In [110]:
import pandas as pd
import LendingClubFunctions as LC
from sklearn.decomposition import PCA
import sklearn as sk
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import grid_search
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import re
import sklearn as sk
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import grid_search
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import nltk
from sklearn import tree
import numpy as np
from sklearn import preprocessing
import numpy as np
import math
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
import random
%pylab inline
pylab.rcParams['figure.figsize']=(8.0,6.0)
from sklearn.ensemble import AdaBoostRegressor

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [3]:
#Define data location
file1='./data/LoanStats3a.csv'
file2='./data/LoanStats3b.csv'
file3='./data/LoanStats3c.csv'
file4='./data/LoanStats3d.csv'
files=[file1,file2,file3,file4]

In [4]:
cols=['loan_amnt','funded_amnt','total_pymnt', 'loan_status', 'int_rate', 'term',
      'desc', 'grade', 'annual_inc', 'purpose', 'emp_length', 'earliest_cr_line', 
      'revol_util', 'home_ownership', 'sub_grade', 'addr_state', 'dti', 'revol_bal', 'installment', 'last_pymnt_d', 'issue_d']

In [5]:
#Load data in from files
df1=LC.GetData(cols, files)
Trouble=[ 'Charged Off', 'Default', 'Does not meet the credit policy.  Status:Charged Off']
Success=['Fully Paid', 'Does not meet the credit policy.  Status:Fully Paid']
df1=LC.CleanData(df1, Success=Success, Trouble=Trouble)

/Users/admin/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [28]:
class LendingClubTransformer(sk.base.BaseEstimator, sk.base.TransformerMixin):
    """
    A transformer that takes as input a dataframe and a cut-off for variance explained. Then 
    performs PCA on the relevant columns and returns a matrix explaining the requested amount
    of variance
    """
    def __init__(self, cols=[], varExp=100):
        self.cols=cols
        self.varExp=varExp
        self.means=[]
        self.stds=[]
        self.pca=PCA()
        self.ElVarExp=None
        pass

    def fit(self, X, y=[]):
        return self.fit_transform(X)
    
    def fit_transform(self, X, y=[]):
        
        self.means=X.describe().ix['mean']
        self.stds=X.describe().ix['std']
        Xs=X[self.cols].dropna().copy()
        for col in Xs.columns:
            Xs[col]= (Xs[col]-self.means[col])/self.stds[col]

        self.pca.fit(Xs)
        el=np.argmax(self.pca.explained_variance_ratio_.cumsum()>self.varExp)
        self.ElVarExp=el
        Xs=self.pca.transform(Xs)[:, :(self.ElVarExp+1)]
        return Xs

    def transform(self, X):
        Xs=X[self.cols].dropna().copy()
        for col in Xs.columns:
            Xs[col]= (Xs[col]-self.means[col])/self.stds[col]
        Xs=self.pca.transform(Xs)[:, :(self.ElVarExp+1)]
        return Xs

In [31]:
class yLendingClubTransformer(sk.base.BaseEstimator, sk.base.TransformerMixin):
    """
    A transformer that takes as input a dataframe and the desired columns as features and returns the requests label
    """
    def __init__(self, cols=[], label=None):
        self.cols=[col for col in cols]
        self.cols.append(label)
        self.label=label
        pass
    

    def transform(self, X):
        return (X[self.cols].dropna())[self.label].as_matrix()

In [271]:
features=['loan_amnt','int_rate','annual_inc','earliest_cr_line', 'revol_util','dti']
Transformer=LendingClubTransformer(cols=features, varExp=.95)
Model=linear_model.Ridge() 

In [280]:
LMCV.predict(pd.DataFrame(np.array([  1.00000000e+04,   1.50000000e-01 ,  1.00000000e+05  , 1.20000000e+02,
   9.00000000e-01 ,  8.40000000e+01]), index=features).T)

array([ 0.75427388])

In [255]:
dfTrain, dfTest = train_test_split(df1[df1.loan_status!=2].dropna(), test_size=0.4)
yTransformer=yLendingClubTransformer(cols=features, label='Fraction_Of_Total')
y=yTransformer.transform(dfTrain)
yTest=yTransformer.transform(dfTest)
Xtrain=Transformer.fit_transform(dfTrain[features].dropna())
Xtest=Transformer.transform(dfTest[features].dropna())

In [256]:
LMCV=grid_search.GridSearchCV(LinearModelPipline , {'Linear__alpha':(1,5, 10, 20, 40, 60, 90),
            'ScalePCA__varExp':( .85, .9, .95), 'ScalePCA__cols':(['loan_amnt','int_rate',
            'annual_inc','earliest_cr_line', 'revol_util','dti'], ['loan_amnt','int_rate','annual_inc'], 
                                ['int_rate','annual_inc', 'dti'])}, cv=5, verbose=True) 

In [258]:
LMCV.fit(dfTrain[features].dropna(), y)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    5.1s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:   18.9s
[Parallel(n_jobs=1)]: Done 315 out of 315 | elapsed:   29.2s finished


Fitting 5 folds for each of 63 candidates, totalling 315 fits


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('ScalePCA', LendingClubTransformer(cols=['loan_amnt', 'int_rate', 'annual_inc', 'earliest_cr_line', 'revol_util', 'dti'],
            varExp=0.95)), ('Linear', Ridge(alpha=40, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, solver='auto', tol=0.001))]),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'ScalePCA__varExp': (0.85, 0.9, 0.95), 'Linear__alpha': (1, 5, 10, 20, 40, 60, 90), 'ScalePCA__cols': (['loan_amnt', 'int_rate', 'annual_inc', 'earliest_cr_line', 'revol_util', 'dti'], ['loan_amnt', 'int_rate', 'annual_inc'], ['int_rate', 'annual_inc', 'dti'])},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=True)

In [259]:
LMCV.best_estimator_

Pipeline(steps=[('ScalePCA', LendingClubTransformer(cols=['loan_amnt', 'int_rate', 'annual_inc', 'earliest_cr_line', 'revol_util', 'dti'],
            varExp=0.95)), ('Linear', Ridge(alpha=90, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, solver='auto', tol=0.001))])

In [262]:
LMCV.score(dfTest[features].dropna(), yTest)

0.14457791697999389

In [263]:
predictions=LMCV.predict(dfTest[features].dropna())

In [264]:
math.sqrt(mean_squared_error(yTest, predictions))

0.2166224248418003

In [276]:
import dill
f=open('LinearModel.txt', 'w')
print >> f, dill.dumps(LMCV.best_estimator_)
f.close()